<a href="https://colab.research.google.com/github/MatteoPao/cubert_keras/blob/main/NeuronPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/MatteoPao/cubert_keras.git

%cd cubert_keras/src/

In [ ]:
from google.colab import drive                                               
drive.mount('/content/drive/')

# %cp -r /content/drive/MyDrive/Tesi_ModelData/cubert_pretrained_model_exceptions /content/cubert_keras/src/fitness/cubert_fitness/cubert_pretrained_model_exceptions
%cp -r /content/drive/MyDrive/Tesi_ModelData/cubert_pretrained_model_variablemisuse /content/cubert_keras/src/fitness/cubert_fitness/cubert_pretrained_model_variablemisuse

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
from intermediate_layer import read_examples_from_json
from keras_bert import load_trained_model_from_checkpoint, get_checkpoint_paths
from cubert.full_cubert_tokenizer import FullCuBertTokenizer, CuBertVariableMisuseProcessor, InputExample
from cubert import tokenizer_registry

import keras
import numpy as np
import tensorflow as tf

In [ ]:
model_path = "fitness/cubert_fitness/cubert_pretrained_model_variablemisuse"
data_path = "../cyclomatic_complexity/final/data_cc.jsontxt"

paths = get_checkpoint_paths(model_path)

examples = read_examples_from_json(data_path, "eval")

tokenizer = FullCuBertTokenizer(code_tokenizer_class=tokenizer_registry.TokenizerEnum.PYTHON.value, vocab_file=paths.vocab)
print("Tokenizer loaded")

model = load_trained_model_from_checkpoint(paths.config, paths.checkpoint, training=True, seq_len=128, out_dim=2)
print("Model loaded")
#model.summary()

layer_name = "NSP-Dense"
layer_name_1 = "Extract"
model = keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
#model.summary()

#tokenizza tutto il dataset
features = tokenizer.convert_examples_to_features(examples, [0, 1], 128)
print("Dataset tokenized")
np.random.shuffle(features)

In [ ]:
inp = []
seg = []
mas = []
label = []

for f in features:
  inp.append(f.input_ids)
  seg.append(f.segment_ids)
  mas.append(f.input_mask)
  label.append(f.label_id)
  
test_input = [np.array(inp), np.array(seg), np.array(mas)]

pred_results = model.predict(test_input)
pred_results = np.transpose(pred_results)

print("Prediction completata")
print("Neuroni: ", pred_results.shape[0])
print("Predictions: ", pred_results.shape[1])

In [ ]:
#Test singolo input
'''
feature = tokenizer.convert_single_example(examples[0], [0, 1], 128)

pred_results = model.predict([np.array(feature.input_ids, ndmin=2), 
                            np.array(feature.segment_ids, ndmin=2), 
                            np.array(feature.input_mask, ndmin=2)])
'''


In [ ]:
threshold_tanh = np.linspace(-1, 1, num=12)[1:11]
label_inv = np.logical_not(label).astype(int)

bestAcc = 0
bestNeu = 0
bestTh = 0
for index, neuron in enumerate(pred_results):
  best_n = 0
  best_t = 0
  for t in threshold_tanh:
    inv = False
    m = tf.keras.metrics.BinaryAccuracy(threshold=t)
    m.update_state(label, neuron)
    acc = m.result().numpy()
    m.reset_state()
    m.update_state(label_inv, neuron)
    acc_inv = m.result().numpy()
    if acc_inv > acc:
      acc = acc_inv
      inv = True
    if acc >= best_n:
      best_n = acc
      best_t = t
      best_inv = inv
  if best_n >= bestAcc:
    print("NEW Best Neuron")
    print("Neuron: %d   Acc: %f   inv: %s" % (index, best_n, best_inv))
    bestAcc = best_n
    bestNeu = index
    bestTh = best_t

print("BEST FINALE")
print("Neurone: ", bestNeu)
print("Accuratezza: ", bestAcc)
print("Threshold: ", bestTh)

print(label)
print(np.where(pred_results[bestNeu] > bestTh, 1, 0))

In [ ]:
%mkdir ../../result
outF = open("../../result/bestNeuron.txt", "x")
outF.write("Neurone: ")
outF.write(str(bestNeu))
outF.write("\nAccuratezza: ")
outF.write(str(bestAcc))
outF.write("\nThreshold: ")
outF.write(str(bestTh))
outF.close()

!zip -r /content/bestNeuron.zip /content/result

from google.colab import files
files.download("/content/bestNeuron.zip")

!rm -r /content/result